In [1]:
from functools import reduce
import numpy as np
import operator

In [2]:
Games = [line.strip() for line in open('p096_sudoku.txt','r').readlines()] #to eliminate '\n'

In [3]:
sep_games  = []
for l in range(1,len(Games),10):
    game = np.zeros((9,9))
    for row in range(l,l+9):
        line = Games[row]
        for col in range(len(line)):
            game[row-l,col] = int(line[col])
    sep_games.append(game)

In [4]:
def check_column_poss(cell,game,poss_nums):
    row_to_check = [r for r in range(9)if r != cell[0]]
    for row in row_to_check:
        if (game[row,cell[1]] != 0) and (game[row,cell[1]] in poss_nums):
            poss_nums.remove(game[row,cell[1]])
            #print(game[row,cell[1]],[row,cell[1]])
    
    return poss_nums

In [5]:
def check_column_unique(cell,game):
    row_to_check = [r for r in range(9)if r != cell[0]]
    nums_around = ''
    for row in row_to_check:
        nums_around += str(game[row,cell[1]])
    
    unique = set(str(game[cell[0],cell[1]])) - set(nums_around)
    if len(unique) == 1:
        game[cell[0],cell[1]] = int(list(unique)[0])
        game = get_possibilities(game)
        return game
    return game

In [6]:
def check_row_poss(cell,game,poss_nums):
    cols_to_check = [c for c in range(9)if c != cell[1]]
    for col in cols_to_check:
        if (game[cell[0],col] != 0) and (game[cell[0],col] in poss_nums):
            poss_nums.remove(game[cell[0],col])
            #print(game[cell[0],col],[cell[0],col])
        
    return poss_nums

In [7]:
def check_row_unique(cell,game):
    cols_to_check = [c for c in range(9)if c != cell[1]]
    nums_around = ''
    for col in cols_to_check:
        nums_around += str(game[cell[0],col])
    unique = set(str(game[cell[0],cell[1]])) - set(nums_around)
    if len(unique) == 1:
        game[cell[0],cell[1]] = int(list(unique)[0])
        game = get_possibilities(game)
        return game
    return game

In [8]:
def chec_square_poss(cell,game,poss_nums):
    cell_position = (cell[0]//3,cell[1]//3)
    corner_cell = [0,0]
    for p in range(len(cell_position)):
        if cell_position[p] == 0:
            corner_cell[p] = cell_position[p]
        elif cell_position[p] == 1:
            corner_cell[p] = cell_position[p]+2
        else : # = 2
            corner_cell[p] = cell_position[p]+4
    
    #after detectecting the corner cell of the square
    #let's reduce the possible numbers
    for r in range(3):
        for c in range(3):
            if (game[corner_cell[0]+r,corner_cell[1]+c] != 0) and (game[corner_cell[0]+r,corner_cell[1]+c] in poss_nums):
                poss_nums.remove(game[corner_cell[0]+r,corner_cell[1]+c])
                #print(game[corner_cell[0]+r,corner_cell[1]+c],[corner_cell[0]+r,corner_cell[1]+c])
    
    return poss_nums

In [9]:
def chec_square_unique(cell,game):
    cell_position = (cell[0]//3,cell[1]//3)
    corner_cell = [0,0]
    for p in range(len(cell_position)):
        if cell_position[p] == 0:
            corner_cell[p] = cell_position[p]
        elif cell_position[p] == 1:
            corner_cell[p] = cell_position[p]+2
        else : # = 2
            corner_cell[p] = cell_position[p]+4
    
    nums_around = ''
    #after detectecting the corner cell of the square
    #let's get the around numbers
    for r in range(3):
        for c in range(3):
            nums_around += str(game[corner_cell[0]+r,corner_cell[1]+c])
    
    unique = set(str(game[cell[0],cell[1]])) - set(nums_around)
    if len(unique) == 1:
        game[cell[0],cell[1]] = int(list(unique)[0])
        game = get_possibilities(game)
        return game
    return game

In [10]:
def get_possibilities(game):
    for row in range(9):
        for col in range(9):
            if (game[row,col] == 0) or (game[row,col] > 9):
                cell = (row,col)
                poss_nums = [i for i in range(1,10)]
                poss_nums = check_column_poss(cell,game,poss_nums)
                poss_nums = check_row_poss(cell,game,poss_nums)
                poss_nums = chec_square_poss(cell,game,poss_nums)
                poss_nums = int(reduce(operator.add,map(str,poss_nums)))
                game[row,col] = poss_nums
                if len(str(poss_nums)) == 1:
                    game = get_possibilities(game)
                
    return game

In [11]:
def get_unique(game):
    for row in range(9):
        for col in range(9):
            cell = (row,col)
            game = check_column_unique(cell,game)
            game = check_row_unique(cell,game)
            game = chec_square_unique(cell,game)
    return game

In [12]:
def no_guessing(game):
    game = game.astype(int)
    last_game = game.copy()
    game = get_possibilities(game)
    while (last_game != game).any():
        last_game = game.copy()
        game = get_unique(game)
        
    return game

In [ ]:
def check_column_correct(cell,game,poss_nums):
    for row in range(9):
        if game[row,cell[1]] in poss_nums:
            poss_nums.remove(game[row,cell[1]])
            #print(game[row,cell[1]],[row,cell[1]])
    
    return len(poss_nums) == 0

In [ ]:
def check_row_correct(cell,game,poss_nums):
    for col in range(9):
        if (game[cell[0],col] in poss_nums):
            poss_nums.remove(game[cell[0],col])
            #print(game[cell[0],col],[cell[0],col])
        
    return len(poss_nums) == 0

In [ ]:
def chec_square_correct(cell,game,poss_nums):
    cell_position = (cell[0]//3,cell[1]//3)
    corner_cell = [0,0]
    for p in range(len(cell_position)):
        if cell_position[p] == 0:
            corner_cell[p] = cell_position[p]
        elif cell_position[p] == 1:
            corner_cell[p] = cell_position[p]+2
        else : # = 2
            corner_cell[p] = cell_position[p]+4
    
    #after detectecting the corner cell of the square
    #let's reduce the possible numbers
    for r in range(3):
        for c in range(3):
            if game[corner_cell[0]+r,corner_cell[1]+c] in poss_nums:
                poss_nums.remove(game[corner_cell[0]+r,corner_cell[1]+c])
                #print(game[corner_cell[0]+r,corner_cell[1]+c],[corner_cell[0]+r,corner_cell[1]+c])
    
    return len(poss_nums) == 0

In [ ]:
def is_correct(game):
    for row in range(9):
        for col in range(9):
            cell = (row,col)
            poss_nums = [i for i in range(1,10)]
            cond_1 = check_column_correct(cell,game,poss_nums)
            cond_2 = check_row_correct(cell,game,poss_nums)
            cond_3 = chec_square_correct(cell,game,poss_nums)
            if not(cond_1 & cond_2 & cond_3):
                return False
    return True

In [13]:
def sol_sudoku(Games):
    for g in range(len(Games)):
        print('=======================================')
        decision   = 't'   
        print('sudoku' + str(g+1) + ' : ')
        game = Games[g]
        print(game)
        print('----------------------------------------')
        while decision == 't': #try again
            if not 0 in game:
                row = int(input('enter the row : '))
                col = int(input('enter the column : '))
                guess = int(input('enter your guess : '))
                game[row,col] = guess
                
            game = no_guessing(game)
            print(game)
            decision = input("So what's next? : ")
       # if not is_correct(game):   
        with open('Answers.txt','a') as Answers:
            Answers.write(reduce(operator.add,map(str,game[0,:3]))+'\n')   
      #  else:
           # print("there's something wrong")
           # break

In [ ]:
sol_sudoku(sep_games)

In [ ]:
sum(list(map(int,list(open('Answers.txt','r')))))